# Chargement des modèles

In [ ]:
from keras.models import model_from_json

In [2]:
# Charge un modèle Keras à partir de son fichier json et h5
def model_loader(json_filepath, h5_filepath):    
    json_file = open(json_filepath, 'r')
    json_model = json_file.read()
    json_file.close()
    
    model = model_from_json(json_model)
    model.load_weights(h5_filepath)
    
    return model

In [ ]:
image_model = model_loader('modèles/image_model.json', 'modèles/image_model.h5')
text_model = model_loader('modèles/text_model.json', 'modèles/text_model.h5')
audio_model = model_loader('modèles/song_model.json', 'modèles/song_model.h5')

# Fonctions de traitement des images

In [4]:
import os
import numpy as np
import cv2

import mediapipe as mp

from moviepy.editor import VideoFileClip

In [5]:
# Dimension des images pour être traitées par le modèle
IMG_WIDTH = 48
IMG_HEIGHT = 48

In [6]:
# Prédit l'émotion exprimée dans une image avec un modèle donné
def predict_face_emotion(model, img):
    # Listes des classes récupérées à partir du modèle
    classes_name = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']
    
    # Image en niveaux de gris
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # Dimension de sortie souhaitée : (1, 48, 48, 1)
    img_processed = np.expand_dims(img_gray, axis=(0, 3))
    # Normalisation de la valeur des pixels dans [0, 1]
    img_processed = img_processed / 255.0

    # Prédiction du modèle sur l'image
    emotion_scores = model.predict(img_processed)

    # Récupération de l'indice de la valeur la plus élevée dans le vecteur de prédiction
    predicted_emotion_index = np.argmax(emotion_scores)

    # Récupération du nom de l'émotion
    predicted_emotion_label = classes_name[predicted_emotion_index]

    return predicted_emotion_index, predicted_emotion_label, emotion_scores

In [7]:
# Affiche sur une image le visage détecté et l'émotion associée
def process_image(frame, face_detection, model):
    # Dimension de la frame
    h, w, _ = frame.shape
    
    # Pixels au format RGB
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    output = face_detection.process(img_rgb)
    
    # Visages détectés sur l'image
    if output.detections is not None:
        for detection in output.detections:
            location_data = detection.location_data
            rbb = location_data.relative_bounding_box
            
            # Récupération des coordonnées du visage détecté
            x, y, width, height = rbb.xmin, rbb.ymin, rbb.width, rbb.height
            
            x = int(x * w)
            y = int(y * h)
            width = int(width * w)
            height = int(height * h)
            
            face_region = frame[y: y + height, x: x + width]
            # Visage sur 48x48 pixels
            face_resized = cv2.resize(face_region, (IMG_WIDTH, IMG_HEIGHT), interpolation=cv2.INTER_AREA)
            
            # Détection de l'émotion par le modèle
            predicted_index, predicted_label, scores = predict_face_emotion(model, face_resized)
            
            # Ajout de l'émotion sur la frame
            text_printed = predicted_label + ' : ' + str(np.round(scores[0][predicted_index], 3))
            cv2.putText(frame, text_printed, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (36, 255, 12), 2)
            
            # Ajout d'une rectangle autour du visage détecté
            cv2.rectangle(frame, (x, y), (x + width, y + height), (36, 255, 12), 2)
            
    return frame

In [8]:
def process_video(input_video_filepath, output_video_filepath, model):
    temp_filepath = 'temp/temp_videofile.mp4'
    
    # Chargement de la vidéo, de l'audio et récupération des FPS de la vidéo
    clip = VideoFileClip(input_video_filepath)
    audio = clip.audio
    fps = clip.fps
    
    # Détection des visages avec une confiance de 50%
    mp_face_detection = mp.solutions.face_detection
    with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.5) as face_detection:
        cap = cv2.VideoCapture(input_video_filepath)
        
        # Récupération et écriture de l'image
        ret, frame = cap.read()
        output_video = cv2.VideoWriter(temp_filepath, cv2.VideoWriter_fourcc(*'MP4V'), fps, (frame.shape[1], frame.shape[0]))

        # Traitement de l'image
        while ret:
            frame = process_image(frame, face_detection, model)
            output_video.write(frame)
            ret, frame = cap.read()

        cap.release()
        output_video.release()
        
        # Rajout de l'audio dans la vidéo
        output_video = VideoFileClip(temp_filepath)
        output_video = output_video.set_audio(audio)
        
        # Ajout du son
        output_video.write_videofile(output_video_filepath, codec='libx264', audio_codec='aac', fps=fps)
        
        # Suppression de la vidéo sans son
        if os.path.exists(temp_filepath):
            os.remove(temp_filepath)

# Fonctions de traitement du texte

In [9]:
import speech_recognition as sr
from deep_translator import GoogleTranslator

import pickle
from keras.preprocessing.sequence import pad_sequences

import re
import nltk
from nltk.corpus import stopwords

from moviepy.video.io.ffmpeg_tools import ffmpeg_extract_subclip
from moviepy.editor import VideoFileClip, ImageClip, concatenate_videoclips

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [11]:
# Transcription du texte à partir d'une vidéo
def transcribe_video_audio(video_path, language='fr'):
    transcribed_text = ''
    recognizer = sr.Recognizer()

    # Chargement de la vidéo et extraction du son
    video_clip = VideoFileClip(video_path)
    video_audio = video_clip.audio

    # Conversion au format .wav
    temp_audio_file = "temp/temp_audio.wav"
    video_audio.write_audiofile(temp_audio_file)
    
    # Ouverture du fichier avec SpeechRecognition
    with sr.AudioFile(temp_audio_file) as audio_file:
        audio_data = recognizer.record(audio_file)

        try:
            if language == 'fr':
                transcribed_text = recognizer.recognize_google(audio_data, language='fr-FR')
            else:
                transcribed_text = recognizer.recognize_google(audio_data)
            # print("Transcription de l'audio en texte :\n", transcribed_text)
        except sr.UnknownValueError:
            print("Impossible de récupérer l'audio de la vidéo")
        except sr.RequestError as e:
            print(f"Erreur : {e}")
            
    # Suppression du fichier temporaire
    if os.path.exists(temp_audio_file):
        os.remove(temp_audio_file)
        
    return transcribed_text

In [12]:
# Traduction du français vers l'anglais
def translate_french_to_english(text):
    translated = GoogleTranslator(source='fr', target='en').translate(text)
    return translated

In [13]:
# Retire les caractères spéciaux d'une chaîne de caractères
def remove_special_chars(text):
    return re.sub(r"[^a-zA-Z0-9]", " ", text)

In [14]:
# Passe la casse du texte en minuscule
def to_lowercase(text):
    return text.lower()

In [15]:
# Retire les 'stop words'
def remove_stop_words(text):
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    return ' '.join(filtered_words)

In [16]:
# Normalisation du texte de la même manière que lors de l'entraînement du modèle
def normalize_text(text):
    # Tokenizer utilisé lors de l'entraînement du modèle
    tokenizer_path = 'tokenizer.pkl'
    # Taille maximale d'une phrase
    max_length = 25
    
    # Chargement du tokenizer
    with open(tokenizer_path, 'rb') as file:
        tokenizer = pickle.load(file)
        
    # Nettoyage de la phrase
    clean_sentence = remove_special_chars(text)
    clean_sentence = to_lowercase(clean_sentence)
    clean_sentence = remove_stop_words(clean_sentence)
    
    # Séquençage et tokenization de la phrase
    sequence = tokenizer.texts_to_sequences([text])
    padded_sequence = pad_sequences(sequence, maxlen=max_length)
    
    return padded_sequence

In [17]:
# Prédiction de l'émotion exprimée dans une phrase selon un modèle
def predict_text_emotion(model, text):
    # Noms des classes récupérés d'après le modèle
    classes_name = ['happy', 'sad', 'angry', 'fear', 'love', 'surprise']

    # Prédiction du modèle sur le texte
    emotion_scores = model.predict(text)

    # Récupération de l'indice de la valeur la plus élevée dans le vecteur de prédiction
    predicted_emotion_index = np.argmax(emotion_scores)

    # Récupération du nom de l'émotion
    predicted_emotion_label = classes_name[predicted_emotion_index]

    return predicted_emotion_index, predicted_emotion_label, emotion_scores

In [18]:
# Constitution un dictionnaire associant temps de la vidéo à une émotion
def get_text_emotions_dict(video_path, model, language='fr', segment_duration=5, segmentation_freq=1):
    # segment_duration : les émotions sont prédites sur une séquence de 5 secondes
    # segmentation_freq : les émotions sont prédites sur des fenêtres décalées d'1 seconde
    
    # Dictionnaire des émotions
    emotions_dict = {}
    
    video_duration = VideoFileClip(video_path).duration
    start_time = 0
    
    while start_time + segmentation_freq <= video_duration:
        # Découpage en séquence
        end_time = start_time + segment_duration
        sequence_path = 'temp/temp_sequence.mp4'
        # Extrait de 'segment_duration=5' secondes
        ffmpeg_extract_subclip(video_path, start_time, end_time, targetname=sequence_path)
        
        # Transcription de la partie séquencée
        transcribed_text = transcribe_video_audio(sequence_path, language)
        
        # Traduction du français vers l'anglais si nécessaire
        sequence_text = transcribed_text
        if language == 'fr':
            sequence_text = translate_french_to_english(transcribed_text)
        
        
        # Préparation de la séquence pour le modèle
        sequence_text = normalize_text(sequence_text)
        
        # Prédiction par le modèle
        predicted_index, predicted_label, scores = predict_text_emotion(model, sequence_text)
        
        # Temps de la vidéo associée à l'émotion prédite par le modèle
        emotions_dict[start_time] = [predicted_index, predicted_label, scores]
        # print("La phrase : '", transcribed_text, "' est associée à : ", predicted_label)
        
        start_time += segmentation_freq
        
    # Suppression du fichier de séquence temporaire
    if os.path.exists(sequence_path):
        os.remove(sequence_path)
        
    # Ajout d'un dernier timecode pour couvrir toute la durée de la vidéo
    emotions_dict[video_duration] = [predicted_index, predicted_label, scores]
            
    return emotions_dict

# Fonctions de traitement du son

In [19]:
import librosa

from pydub import AudioSegment

In [26]:
# Traitement de segments audio pour être acceptés en entrée du modèle
def preprocess_audio_segment(segment_path):
    
    # Chargement de l'audio
    _, sr = librosa.load(segment_path)
    raw_audio = AudioSegment.from_file(segment_path)
    
    # Chargement des échantillons sous forme de tableau Numpy
    samples = np.array(raw_audio.get_array_of_samples(), dtype='float32')
    
    # Normalisation des données du son
    trimmed, _ = librosa.effects.trim(samples, top_db=25)
    padded = np.pad(trimmed, (0, 600000 - len(trimmed)), 'constant')
    
    # Récupération des features
    FRAME_LENGTH = 2048
    HOP_LENGTH = 512
    
    zcr = librosa.feature.zero_crossing_rate(y=padded, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
    rms = librosa.feature.rms(y=padded, frame_length=FRAME_LENGTH, hop_length=HOP_LENGTH)
    mfccs = librosa.feature.mfcc(y=padded, sr=sr, n_mfcc=13, hop_length=HOP_LENGTH)
    
    # Concaténation des features dans un tableau Numpy
    processed_segment = np.concatenate((
        np.swapaxes(zcr, 0, 1),
        np.swapaxes(rms, 0, 1),
        np.swapaxes(mfccs, 0, 1)),
        axis=1
    )
    
    processed_segment = processed_segment.astype('float32')
    processed_segment = np.expand_dims(processed_segment, axis=0)
    
    return processed_segment

In [21]:
# Prédiction de l'émotion exprimée dans l'audio selon un modèle
def predict_audio_emotion(model, audio):
    # Noms des classes récupérés d'après le modèle
    classes_name = ['angry', 'fear', 'neutral', 'sad', 'disgust', 'happy']

    # Prédiction du modèle sur le texte
    emotion_scores = model.predict(audio)

    # Récupération de l'indice de la valeur la plus élevée dans le vecteur de prédiction
    predicted_emotion_index = np.argmax(emotion_scores)

    # Récupération du nom de l'émotion
    predicted_emotion_label = classes_name[predicted_emotion_index]

    return predicted_emotion_index, predicted_emotion_label, emotion_scores

In [22]:
def get_audio_emotions_dict(video_path, model, segment_duration=5, segmentation_freq=1):
    # segment_duration : les émotions sont prédites sur une séquence de 5 secondes
    # segmentation_freq : les émotions sont prédites sur des fenêtres décalées d'1 seconde
    
    temp_audio_file = 'temp/temp_audio.wav'
    emotions_dict = {}
    start_time = 0
    video_duration = VideoFileClip(video_path).duration
    
    while start_time + segmentation_freq <= video_duration:
        # Séquençage des audios
        end_time = min(start_time + segment_duration, video_duration)
        video_clip = VideoFileClip(video_path).subclip(start_time, end_time)
        
        # Normalisation de l'audio
        video_clip.audio.write_audiofile(temp_audio_file)
        processed_segment = preprocess_audio_segment(temp_audio_file)
        
        # Prédiction de l'émotion
        predicted_emotion_index, predicted_emotion_label, emotion_scores = predict_audio_emotion(model, processed_segment)
        emotions_dict[start_time] = [predicted_emotion_index, predicted_emotion_label, emotion_scores]
        
        start_time += segmentation_freq
        
    if os.path.exists(temp_audio_file):
        os.remove(temp_audio_file)
        
    emotions_dict[video_duration] = [predicted_emotion_index, predicted_emotion_label, emotion_scores]
    
    return emotions_dict

In [25]:
# Debug
# aed = get_audio_emotions_dict('vidéos/voeux_10sec_low.mp4', audio_model, 1, 1)

MoviePy - Writing audio in temp/temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 1s 1s/step
MoviePy - Writing audio in temp/temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 58ms/step
MoviePy - Writing audio in temp/temp_audio.wav


...

1/1 [==============================] - 0s 49ms/step
MoviePy - Writing audio in temp/temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 45ms/step
MoviePy - Writing audio in temp/temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 53ms/step


# Fonctions annexes

In [27]:
# Création d'une nouvelle vidéo avec les émotions détectées
def add_text_to_video(original_video_path, output_video_path, emotions_dict, label, position=1):
    """Ajoute sur une nouvelle vidéo les émotions détectées selon un dictionnaire{timecode: emotion}.

    Args:
        original_video_path (str): chemin vers la vidéo originale.
        output_video_path (str): chemin de la nouvelle vidéo.
        emotions_dict (dict): dictionnaire associant les temps de la vidéo et l'émotion exprimée.
        label (str): label qui sera inscrit sur la vidéo et associée au score de prédiction.
        position (int, optional): position du label, par défaut 1.
    """
    
    # Chargement de la vidéo
    video = VideoFileClip(original_video_path)
    
    # Transformation en liste des timecodes de la vidéo
    change_times = list(emotions_dict.keys())
    change_times.sort()
    
    # Stockage des clips modifiés dans une liste
    clips_with_text = []

    # Parcours des émotions de la liste
    for i in range(len(change_times) - 1):
        start_time = change_times[i]
        end_time = change_times[i + 1]

        # Récupération des émotions correspondant aux timecodes
        start_emotion = emotions_dict[start_time]

        # Création de sous-clip où l'émotion sera indiquée
        subclip = video.subclip(start_time, end_time)
        subclip_duration = subclip.duration
        
        # Récupération du texte à inscrire dans le sous-clip
        emotion_label = start_emotion[1]
        emotion_proba = np.round(start_emotion[2][0][start_emotion[0]], 5)
        emotion_proba_str = str(emotion_proba)
        text = emotion_label + " : " + emotion_proba_str

        # Ajout du texte dans la vidéo pour toutes les frames (durée du sous-clip * FPS (nombre de frames par sec.))
        for t in np.linspace(0, subclip_duration, int(subclip_duration * video.fps)):

            # Récupération de l'image au temps t
            frame = subclip.get_frame(t)
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR) # format BGR pour modification

            # Ajout du texte sur l'image selon le label et la position
            cv2.putText(frame, f"{label} : {text}", (50, 50 * position), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB) # format RGB fin de modification

            # Création du clip
            text_clip = ImageClip(frame, duration=1 / video.fps)
            clips_with_text.append(text_clip)

    # Concaténation de tous les clips à la fin du traitement
    final_clip = concatenate_videoclips(clips_with_text, method="compose")
    # Rajout de l'audio de la vidéo d'origine
    final_clip = final_clip.set_audio(video.audio)

    # Enregistrement de la vidéo
    final_clip.write_videofile(output_video_path, codec='libx264', fps=video.fps)

In [28]:
def emotion_recognition_video(original_videopath, output_videopath,
                              image_model, text_model, audio_model, language='fr',
                              segment_duration=5, segmentation_freq=1):
    
    # Face Emotion Recognition
    fer_videopath = 'temp/temp_fer.mp4'
    process_video(original_videopath, fer_videopath, image_model)
    
    
    # Text Emotion Recognition
    ter_videopath = 'temp/temp_ter.mp4'
    text_emotions_dict = get_text_emotions_dict(
        fer_videopath,
        text_model,
        language,
        segment_duration=segment_duration,
        segmentation_freq=segmentation_freq
    )
    add_text_to_video(
        original_video_path=fer_videopath,
        output_video_path=ter_videopath,
        emotions_dict=text_emotions_dict,
        label='Text',
        position=1
    )
    
    
    # Speech Emotion Recognition
    audio_emotions_dict = get_audio_emotions_dict(
        ter_videopath,
        audio_model,
        segment_duration=segment_duration,
        segmentation_freq=segmentation_freq
    )
    add_text_to_video(
        original_video_path=ter_videopath,
        output_video_path=output_videopath,
        emotions_dict=audio_emotions_dict,
        label='Audio',
        position=2
    )
    
    # Suppressions des fichiers temporaires
    if os.path.exists(fer_videopath):
        os.remove(fer_videopath)
    
    # Des bugs peuvent survenir avec la suppression du fichier tant que le notebook est ouvert
    # La fonction ffmpeg retient parfois la ressources sur le fichier ter_videopath='temp/temp_ter.mp4'
    # if os.path.exists(ter_videopath):
    #     os.remove(ter_videopath)
    
    print('Vidéo traitée et disponible : ', output_videopath)
    

# Test modalités séparées

## Partie image

In [29]:
# original_videopath = 'vidéos/voeux_10sec.mp4'
# fer_videopath = 'output/fer_video.mp4'
# process_video(original_videopath, fer_videopath, image_model)

## Partie texte

In [30]:
# ter_videopath = 'output/ter_video.mp4'

# text_emotions_dict = get_text_emotions_dict(fer_videopath, text_model)
# add_text_to_video(
#     original_video_path=fer_videopath,
#     output_video_path=ter_videopath,
#     emotions_dict=text_emotions_dict,
#     label='Text',
#     position=1
# )

## Partie audio

In [31]:
# ser_videopath = 'output/ser_video.mp4'

# audio_emotions_dict = get_audio_emotions_dict('vidéos/voeux_10sec_low.mp4', audio_model)

In [32]:
# add_text_to_video(
#     original_video_path=ter_videopath,
#     output_video_path=ser_videopath,
#     emotions_dict=audio_emotions_dict,
#     label='Audio',
#     position=2
# )

# Test ensemble des modalités

In [33]:
emotion_recognition_video('vidéos/voeux_10sec.mp4', 'output/final_video.mp4', image_model, text_model, audio_model)

1/1 [==============================] - 0s 34ms/step
Moviepy - Building video temp/temp_fer.mp4.
MoviePy - Writing audio in temp_ferTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
Moviepy - Writing video temp/temp_fer.mp4



Moviepy - Done !
Moviepy - video ready temp/temp_fer.mp4
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
MoviePy - Writing audio in temp/temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 1s 502ms/step
Moviepy - Running:
>>> "+ " ".join(cmd)
Moviepy - Command successful
MoviePy - Writing audio in temp/temp_audio.wav


...

1/1 [==============================] - 0s 130ms/step
MoviePy - Writing audio in temp/temp_audio.wav


MoviePy - Done.


1/1 [==============================] - 0s 138ms/step
Moviepy - Building video output/final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video output/final_video.mp4



Moviepy - Done !
Moviepy - video ready output/final_video.mp4
Vidéo traitée et disponible :  output/final_video.mp4
